In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

def create_optimized_silver_spark_session():
    """
    Crea una sesión de Spark optimizada para recursos con configuración robusta
    """
    try:
        spark = (
            SparkSession.builder
            .appName("SilverLayer-Optimized")
            .config('spark.jars.packages', 
                    'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2,'
                    'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.96.1,'
                    'org.apache.hadoop:hadoop-aws:3.3.4,'
                    'software.amazon.awssdk:bundle:2.20.18')  # Added for better S3 stability
            # Configuración Nessie/Iceberg
            .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
            .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
            .config("spark.sql.catalog.nessie.ref", "main")
            .config("spark.sql.catalog.nessie.authentication.type", "NONE")
            .config("spark.sql.catalog.nessie.warehouse", "s3a://lakehouse/")
            .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
            .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
            
            # Configuración S3/MinIO optimizada
            .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
            .config("spark.sql.catalog.nessie.s3.access-key-id", "minioadmin")
            .config("spark.sql.catalog.nessie.s3.secret-access-key", "minioadmin")
            .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
            
            # Configuración Hadoop/S3A optimizada
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
            .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
            .config("spark.hadoop.fs.s3a.path.style.access", "true")
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
            .config("spark.hadoop.fs.s3a.connection.maximum", "100")
            .config("spark.hadoop.fs.s3a.attempts.maximum", "10")
            .config("spark.hadoop.fs.s3a.retry.limit", "5")
            
            # Extensiones
            .config("spark.sql.extensions", 
                   "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,"
                   "org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
            
            # OPTIMIZACIONES DE MEMORIA Y RECURSOS
            .config("spark.driver.memory", "4g")           # Aumentado para estabilidad
            .config("spark.executor.memory", "4g")         # Aumentado para operaciones Iceberg
            .config("spark.memory.fraction", "0.8")        # Porcentaje de memoria para ejecución
            .config("spark.memory.storageFraction", "0.3") # Memoria para storage
            .config("spark.sql.adaptive.enabled", "true")  # Query execution adaptativo
            .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
            .config("spark.sql.adaptive.skewJoin.enabled", "true")
            
            # OPTIMIZACIONES PARA ICEBERG
            .config("spark.sql.iceberg.handle-timestamp-without-timezone", "true")
            .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
            .config("spark.sql.sources.partitionOverwriteMode", "dynamic")
            .config("spark.sql.catalog.nessie.vectorization-enabled", "false")  # Mejor estabilidad
            
            # GESTIÓN DE CACHE Y SERIALIZACIÓN
            .config("spark.sql.inMemoryColumnarStorage.compressed", "true")
            .config("spark.sql.inMemoryColumnarStorage.batchSize", "10000")
            .config("spark.sql.parquet.compression.codec", "snappy")
            
            # MANEJO DE ERRORES Y RECONEXIÓN
            .config("spark.sql.retainGroupColumns", "false")
            .config("spark.cleaner.periodicGC.interval", "1min")
            .config("spark.cleaner.referenceTracking.cleanCheckpoints", "true")
            
            .getOrCreate()
        )
        
        # Configuración adicional vía SparkContext
        spark.sparkContext.setLogLevel("WARN")  # Reducir verbosidad
        
        print("✅ Sesión Spark optimizada creada exitosamente")
        return spark
        
    except Exception as e:
        print(f"❌ Error creando sesión Spark: {e}")
        raise

# Crear sesión optimizada
spark = create_optimized_silver_spark_session()

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
software.amazon.awssdk#bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b365c5e4-ee2e-4e30-93c6-e25557c0bf32;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.96.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found software.amazon.awssdk#bundle;2.20.18 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
downloading https://repo1

✅ Sesión Spark optimizada creada exitosamente


In [ ]:
import re
from pyspark.sql.functions import (
    lit, col, when, length, to_timestamp, year, month, dayofmonth,
    datediff, current_date, udf
)
from pyspark.sql.types import StringType
from datetime import datetime

# =========================
# Normalización y validación
# =========================
def _normalize_cols(df):
    import re
    new_cols = []
    for c in df.columns:
        # camelCase / PascalCase → snake_case
        c2 = re.sub(r'(?<!^)(?=[A-Z])', '_', c)
        c2 = c2.replace('-', '').replace(' ', '').lower()

        # normalizaciones manuales más comunes según tus archivos
        c2 = (
            c2.replace('userid', 'user_id')
               .replace('postid', 'post_id')
               .replace('creationdate', 'creation_date')
               .replace('displayname', 'display_name')
               .replace('accountid', 'account_id')
               .replace('userdisplayname', 'user_display_name')
        )
        new_cols.append(c2)
    return df.toDF(*new_cols)


def _validate_schema(df, dataset_name, year, bronze_path):
    cols = set(df.columns)
    ds = (dataset_name or "").lower()

    if ds == "comments":
        required = {"id", "post_id", "text", "user_id", "creation_date"}
    elif ds == "posts":
        required = {"id", "post_type_id", "creation_date"}
    elif ds == "badges":
        required = {"id", "user_id", "name", "date"}
    elif ds == "users":
        required = {"id", "display_name", "creation_date"}
    else:
        return

    missing = required - cols
    if missing:
        # si faltan, intenta ver si hay equivalentes antes de fallar
        alt_map = {
            "user_id": ["userid"],
            "post_id": ["postid"],
            "creation_date": ["creationdate"],
            "display_name": ["displayname"],
        }
        for req, alts in alt_map.items():
            if req in missing and any(a in cols for a in alts):
                missing.remove(req)

    if missing:
        sample_cols = ", ".join(sorted(list(cols))[:20])
        raise ValueError(
            f"[{dataset_name} {year}] El archivo no parece de '{dataset_name}'. "
            f"Faltan columnas: {sorted(list(missing))}. "
            f"Vistas (muestra): {sample_cols} ... (path: {bronze_path})"
        )


# =========================
# Hadoop FS helpers (path-aware) → evita Wrong FS file:///
# =========================
def _hconf():
    return spark._jsc.hadoopConfiguration()

def _jPath(p: str):
    return spark._jvm.org.apache.hadoop.fs.Path(p)

def _fs_for(p: str):
    # * clave: usa el FileSystem del Path (respeta esquema s3a://) *
    return _jPath(p).getFileSystem(_hconf())

def _exists(p: str) -> bool:
    try:    return _fs_for(p).exists(_jPath(p))
    except: return False

def _is_dir(p: str) -> bool:
    try:    return _fs_for(p).isDirectory(_jPath(p))
    except: return False

def _listdir(p: str):
    try:    return _fs_for(p).listStatus(_jPath(p))
    except: return []

def _parquet_files_in(dir_str: str, limit: int = None):
    out = []
    for st in _listdir(dir_str):
        pp = st.getPath().toString()
        if st.isFile() and pp.endswith(".parquet"):
            out.append(pp)
    if limit: out = out[:limit]
    return out

# =========================
# Resolver robusto de rutas (ajustado a tu bucket)
# =========================
def _resolve_manual_bronze_path(dataset_name, year=None):
    ds = dataset_name.lower()
    base = "s3a://lakehouse/bronze"

    if year is None:
        # badges/users en archivo único
        candidate_patterns = [
            f"{base}/{ds}.parquet",              # badges.parquet / users.parquet
            f"{base}/{ds.capitalize()}.parquet", # por si viene con mayúscula inicial
        ]
    else:
        # 🔹 Permitir variantes singulares/plurales
        if ds.endswith("s"):
            ds_singular = ds[:-1]
        else:
            ds_singular = ds

        # 🔹 Construcción de patrones posibles
        candidate_patterns = [
            f"{base}/{ds.capitalize()}_{year}.parquet",
            f"{base}/{ds.title()}_{year}.parquet",
            f"{base}/{ds}_{year}.parquet",
            f"{base}/{ds}_{year}/",
            # 👇 añade equivalentes singulares
            f"{base}/{ds_singular.capitalize()}_{year}.parquet",
            f"{base}/{ds_singular}_{year}.parquet",
            f"{base}/{ds_singular}_{year}/",
        ]

    # columnas esperadas para validación rápida
    expected = {
        "comments": {"id", "post_id", "text", "user_id", "creation_date"},
        "posts":    {"id", "post_type_id", "creation_date"},
        "badges":   {"id", "user_id", "name", "date"},
        "users":    {"id", "display_name", "creation_date"},
    }.get(ds, set())

    tried = []
    for cand in candidate_patterns:
        if not _exists(cand):
            tried.append(f"(no existe) {cand}")
            continue

        if _is_dir(cand):
            # carpeta (ej: comments_2021/) → busca .parquet dentro
            parquet_list = _parquet_files_in(cand, limit=50)
            if not parquet_list:
                tried.append(f"(carpeta sin .parquet) {cand}")
                continue

            for pq in parquet_list:
                try:
                    df_try = spark.read.parquet(pq)
                    df_try = _normalize_cols(df_try)
                    cols = set(df_try.columns)
                    if expected.issubset(cols):
                        _validate_schema(df_try, ds, year, pq)
                        return pq
                    else:
                        tried.append(f"(mismatch schema en {pq}, cols: {sorted(list(cols))[:12]})")
                except Exception as e:
                    tried.append(f"(error leyendo {pq}: {e})")
        else:
            # archivo directo
            try:
                df_try = spark.read.parquet(cand)
                df_try = _normalize_cols(df_try)
                cols = set(df_try.columns)
                if expected.issubset(cols):
                    _validate_schema(df_try, ds, year, cand)
                    return cand
                else:
                    tried.append(f"(mismatch schema en {cand}, cols: {sorted(list(cols))[:12]})")
            except Exception as e:
                tried.append(f"(error leyendo {cand}: {e})")

    details = "\n  - ".join(tried) if tried else "(sin intentos)"
    raise FileNotFoundError(
        f"No pude resolver una ruta válida para dataset='{dataset_name}', year={year} en {base}.\n"
        f"Intentos:\n  - {details}\n"
        f"Verifica nombres y que existan los archivos en el bucket."
    )


# =========================
# Lector principal
# =========================
def read_bronze_data(source_type="manual", dataset_name=None, year=None, limit=None):
    """
    Lee datos Bronze desde S3A (MinIO/AWS), normaliza columnas y valida esquema.
    """
    ds = (dataset_name or "").lower()

    if source_type == "manual":
        if ds in ["comments", "post", "posts"] and year:
            bronze_path = _resolve_manual_bronze_path(ds, year=year)
        elif ds in ["badges", "users"] and year is None:
            bronze_path = _resolve_manual_bronze_path(ds, year=None)
        else:
            raise ValueError(f"Combinación no válida: dataset={dataset_name}, year={year}")

    elif source_type == "dlt":
        if ds == "comments" and year == 2021:
            base_dir = "s3a://lakehouse/bronze/comments_2021/comments_2021/"
            if not _exists(base_dir) or not _is_dir(base_dir):
                raise FileNotFoundError(f"No existe carpeta DLT: {base_dir}")
            cands = _parquet_files_in(base_dir, limit=100)
            if not cands:
                raise FileNotFoundError(f"No hay .parquet dentro de {base_dir}")
            bronze_path = None
            last_errs = []
            for pq in cands:
                try:
                    df_try = spark.read.parquet(pq)
                    df_try = _normalize_cols(df_try)
                    _validate_schema(df_try, ds, year, pq)
                    bronze_path = pq
                    print(f"✅ Encontrado archivo DLT: {pq.split('/')[-1]}")
                    break
                except Exception as e:
                    last_errs.append(str(e))
            if bronze_path is None:
                raise ValueError(f"No se halló parquet válido en {base_dir}. Errores: {last_errs[:3]}")
        else:
            raise ValueError("DLT solo disponible para comments 2021 en este flujo.")
    else:
        raise ValueError("source_type debe ser 'manual' o 'dlt'")

    print(f"Leyendo de: {bronze_path}")
    try:
        df = spark.read.parquet(bronze_path)
        df = _normalize_cols(df)
        print(f"📑 Columnas normalizadas: {df.columns}")
        _validate_schema(df, dataset_name, year, bronze_path)
        if limit:
            df = df.limit(limit)
        print(f"✅ Leídos {df.count()} registros de {bronze_path}")
        return df
    except Exception as e:
        print(f"❌ Error leyendo {bronze_path}: {e}")
        raise


# =========================
# UDF para decodificar binarios
# =========================
def binary_to_utf8(binary_data):
    try:
        if binary_data is None:
            return None
        return binary_data.decode('utf-8')
    except Exception as e:
        return f"[DECODE_ERROR: {str(e)}]"

# ⚠ CORRECCIÓN: Faltaba el paréntesis de cierre
binary_to_utf8_udf = udf(binary_to_utf8, StringType())


# =========================
# Setup Nessie
# =========================
def setup_nessie_namespaces():
    """Crear los namespaces necesarios en Nessie"""
    print("=== CONFIGURANDO NAMESPACES EN NESSIE ===")
    
    try:
        print("Namespaces existentes:")
        spark.sql("SHOW NAMESPACES IN nessie").show()
    except Exception as e:
        print(f"Error mostrando namespaces: {e}")
    
    try:
        print("Creando namespace 'silver'...")
        spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.silver")
        print("✅ Namespace 'silver' creado exitosamente")
    except Exception as e:
        print(f"Error creando namespace silver: {e}")
        try:
            spark.sql("CREATE SCHEMA IF NOT EXISTS nessie.silver")
            print("✅ Schema 'silver' creado exitosamente")
        except Exception as e2:
            print(f"Error creando schema: {e2}")
    
    try:
        print("Namespaces después de la creación:")
        spark.sql("SHOW NAMESPACES IN nessie").show()
    except Exception as e:
        print(f"Error verificando namespaces: {e}")


# =========================
# Schema vacío para comments
# =========================
def create_empty_comments_df():
    """Schema actualizado para comments con TODAS las columnas"""
    from pyspark.sql.types import StructType, StructField, LongType, StringType, IntegerType, BooleanType, TimestampType
    
    schema = StructType([
        # Bronze (7)
        StructField("comment_id", LongType(), True),
        StructField("post_id", LongType(), True),
        StructField("score", LongType(), True),
        StructField("comment_text", StringType(), True),
        StructField("creation_date", TimestampType(), True),
        StructField("user_id", LongType(), True),
        StructField("user_display_name", StringType(), True),
        # Enriquecimiento (7)
        StructField("score_category", StringType(), True),
        StructField("text_length", IntegerType(), True),
        StructField("comment_year", IntegerType(), True),
        StructField("comment_month", IntegerType(), True),
        StructField("comment_day", IntegerType(), True),
        StructField("has_user_display_name", BooleanType(), True),
        StructField("load_date", TimestampType(), True)
    ])
    
    return spark.createDataFrame([], schema)


# =========================
# TRANSFORMACIONES SILVER
# =========================

def transform_comments_to_silver_with_year(df, year, load_timestamp):
    """Transforma comments con TODAS las columnas (7 Bronze + 7 Enriquecimiento)"""
    return (
        df
        .withColumnRenamed("id", "comment_id")
        .withColumnRenamed("text", "comment_text_binary")
        .withColumnRenamed("user_display_name", "user_display_name_binary")
        .withColumn("comment_text", binary_to_utf8_udf(col("comment_text_binary")))
        .withColumn("user_display_name", binary_to_utf8_udf(col("user_display_name_binary")))
        .withColumn("score_category",
                   when(col("score") >= 5, "high")
                   .when(col("score") >= 1, "medium")
                   .otherwise("low"))
        .withColumn("text_length", 
                   when(col("comment_text").isNotNull(), length(col("comment_text")))
                   .otherwise(0))
        .withColumn("has_user_display_name", 
                   when(col("user_display_name").isNull() | 
                        (col("user_display_name") == ""), 
                        False).otherwise(True))
        .withColumn("comment_year", lit(year))
        .withColumn("comment_month", month(to_timestamp(col("creation_date"))))
        .withColumn("comment_day", dayofmonth(to_timestamp(col("creation_date"))))
        .withColumn("load_date", lit(load_timestamp).cast("timestamp"))
        .withColumn("is_text_decoded", ~col("comment_text").contains("[DECODE_ERROR]"))
        .filter(col("is_text_decoded") == True)
        .select(
            "comment_id", "post_id", "score", "comment_text", "creation_date",
            "user_id", "user_display_name", "score_category", "text_length",
            "comment_year", "comment_month", "comment_day",
            "has_user_display_name", "load_date"
        )
    )


def transform_badges_to_silver(badges_df, load_timestamp):
    """Transforma badges con TODAS las columnas (6 Bronze + 3 Enriquecimiento)"""
    return (
        badges_df
        .withColumnRenamed("id", "badge_id")
        .withColumnRenamed("name", "badge_name")
        .withColumnRenamed("date", "award_date")
        .withColumnRenamed("class", "badge_class")
        .withColumnRenamed("tag_based", "is_tag_based")
        .withColumn("badge_year", year(to_timestamp(col("award_date"))))
        .withColumn("badge_month", month(to_timestamp(col("award_date"))))
        .withColumn("load_date", lit(load_timestamp).cast("timestamp"))
        .withColumn("badge_name", binary_to_utf8_udf(col("badge_name")))
        .select(
            "badge_id", "user_id", "badge_name", "award_date",
            "badge_class", "is_tag_based", "badge_year", "badge_month", "load_date"
        )
    )


def transform_users_to_silver(users_df, load_timestamp):
    """Transforma users con TODAS las columnas (12 Bronze + 3 Enriquecimiento)"""
    return (
        users_df
        .withColumnRenamed("id", "user_id")
        .withColumn("user_age_days", 
                   datediff(current_date(), to_timestamp(col("creation_date"))))
        .withColumn("is_active", 
                   datediff(current_date(), to_timestamp(col("last_access_date"))) <= 365)
        .withColumn("load_date", lit(load_timestamp).cast("timestamp"))
        .withColumn("display_name", binary_to_utf8_udf(col("display_name")))
        .withColumn("about_me", binary_to_utf8_udf(col("about_me")))
        .withColumn("website_url", binary_to_utf8_udf(col("website_url")))
        .withColumn("location", binary_to_utf8_udf(col("location")))
        .select(
            "user_id", "reputation", "creation_date", "display_name",
            "last_access_date", "about_me", "views", "up_votes", "down_votes",
            "website_url", "location", "account_id",
            "user_age_days", "is_active", "load_date"
        )
    )


def transform_posts_to_silver(posts_df, year, load_timestamp):
    """Transforma posts con TODAS las columnas (21 Bronze + 4 Enriquecimiento)"""
    return (
        posts_df
        .withColumnRenamed("id", "post_id")
        .withColumn("post_year", lit(year))
        .withColumn("post_month", month(to_timestamp(col("creation_date"))))
        .withColumn("post_day", dayofmonth(to_timestamp(col("creation_date"))))
        .withColumn("load_date", lit(load_timestamp).cast("timestamp"))
        .withColumn("body", binary_to_utf8_udf(col("body")))
        .withColumn("title", binary_to_utf8_udf(col("title")))
        .withColumn("tags", binary_to_utf8_udf(col("tags")))
        .withColumn("owner_display_name", binary_to_utf8_udf(col("owner_display_name")))
        .withColumn("last_editor_display_name", binary_to_utf8_udf(col("last_editor_display_name")))
        .withColumn("content_license", binary_to_utf8_udf(col("content_license")))
        .select(
            "post_id", "post_type_id", "accepted_answer_id", "creation_date",
            "score", "view_count", "body", "owner_user_id", "owner_display_name",
            "last_editor_user_id", "last_editor_display_name", "last_edit_date",
            "last_activity_date", "title", "tags", "answer_count", "comment_count",
            "favorite_count", "content_license", "parent_id", "community_owned_date",
            "closed_date", "post_year", "post_month", "post_day", "load_date"
        )
    )


# =========================
# PROCESAMIENTO
# =========================

def transform_multiple_years():
    """Transforma datos de comments de múltiples fuentes y años"""
    limit_per_source = 1000
    current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    all_comments = None
    
    data_sources = [
        {"type": "manual", "dataset": "comments", "year": 2020},
        {"type": "dlt", "dataset": "comments", "year": 2021},
    ]
    
    for source in data_sources:
        print(f"Procesando: {source['type']} - {source['dataset']} {source['year']}...")
        try:
            source_data = read_bronze_data(
                source_type=source['type'],
                dataset_name=source['dataset'], 
                year=source['year'],
                limit=limit_per_source
            )
            if source_data.count() > 0:
                source_transformed = transform_comments_to_silver_with_year(
                    source_data, source['year'], current_timestamp
                )
                if all_comments is None:
                    all_comments = source_transformed
                else:
                    all_comments = all_comments.union(source_transformed)
                print(f"  ✅ {source['type']} {source['year']}: {source_transformed.count()} registros")
            else:
                print(f"  ⚠  {source['type']} {source['year']}: Sin datos")
        except Exception as e:
            print(f"  ❌ Error procesando {source['type']} {source['year']}: {e}")
    
    if all_comments is None:
        print("⚠  No se pudieron procesar fuentes, creando DataFrame vacío")
        all_comments = create_empty_comments_df()
    
    return all_comments


def merge_into_silver_table_compliant(silver_df, table_name, key_columns):
    """MERGE real que maneja duplicados y preserva históricos"""
    silver_table_path = f"nessie.silver.{table_name}"
    print(f"Realizando MERGE en: {silver_table_path}")
    
    try:
        spark.sql(f"DESCRIBE {silver_table_path}").show()
        table_exists = True
        print(f"✅ Tabla {silver_table_path} existe")
    except:
        table_exists = False
        print(f"ℹ  Tabla {silver_table_path} no existe, se creará")
    
    if not table_exists:
        print(f"Creando nueva tabla Iceberg: {silver_table_path}")
        (silver_df
         .writeTo(silver_table_path)
         .using("iceberg")
         .tableProperty("format-version", "2")
         .tableProperty("write.update.mode", "merge-on-read")
         .tableProperty("write.merge.mode", "merge-on-read")
         .create())
        print(f"✅ Tabla creada con {silver_df.count()} registros iniciales")
    else:
        print(f"Realizando MERGE...")
        silver_df.createOrReplaceTempView("new_data")
        join_condition = ' AND '.join([f'target.{col} = source.{col}' for col in key_columns])
        merge_sql = f"""
        MERGE INTO {silver_table_path} AS target
        USING new_data AS source
        ON {join_condition}
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
        """
        spark.sql(merge_sql)
        print("✅ MERGE completado")
        final_count = spark.sql(f"SELECT COUNT(*) as total FROM {silver_table_path}").collect()[0]['total']
        print(f"📊 Total registros: {final_count}")
    
    print(f"\n📸 Snapshots de {table_name}:")
    snapshots_df = spark.sql(f"""
        SELECT snapshot_id, committed_at, operation, 
               summary['added-records'] as added_records,
               summary['deleted-records'] as deleted_records
        FROM {silver_table_path}.snapshots 
        ORDER BY committed_at DESC
    """)
    snapshots_df.show(truncate=False)
    return snapshots_df.count()


def transform_other_datasets():
    """Transforma badges, users, posts a Silver"""
    current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    try:
        print("Procesando badges...")
        badges_df = read_bronze_data(source_type="manual", dataset_name="badges", limit=1000)
        silver_badges = transform_badges_to_silver(badges_df, current_timestamp)
        merge_into_silver_table_compliant(silver_badges, "badges", ["badge_id"])
    except Exception as e:
        print(f"❌ Error badges: {e}")
    
    try:
        print("Procesando users...")
        users_df = read_bronze_data(source_type="manual", dataset_name="users", limit=1000)
        silver_users = transform_users_to_silver(users_df, current_timestamp)
        merge_into_silver_table_compliant(silver_users, "users", ["user_id"])
    except Exception as e:
        print(f"❌ Error users: {e}")
    
    try:
        print("Procesando posts...")
        posts_years = [2020, 2021]
        all_posts = None
        for year in posts_years:
            try:
                posts_df = read_bronze_data(source_type="manual", dataset_name="posts", year=year, limit=1000)
                posts_transformed = transform_posts_to_silver(posts_df, year, current_timestamp)
                if all_posts is None:
                    all_posts = posts_transformed
                else:
                    all_posts = all_posts.union(posts_transformed)
            except Exception as e:
                print(f"  ❌ Error posts {year}: {e}")
        if all_posts and all_posts.count() > 0:
            merge_into_silver_table_compliant(all_posts, "posts", ["post_id"])
    except Exception as e:
        print(f"❌ Error posts: {e}")


def verify_silver_compliance():
    """Verifica cumplimiento de requisitos Silver"""
    print("=== VERIFICACIÓN SILVER ===")
    for table in ["comments", "badges", "users", "posts"]:
        print(f"\n📋 {table}:")
        try:
            spark.sql(f"DESCRIBE nessie.silver.{table}").show()
            snapshots = spark.sql(f"SELECT COUNT(*) as c FROM nessie.silver.{table}.snapshots").collect()
            schema = spark.sql(f"DESCRIBE nessie.silver.{table}").collect()
            count = spark.sql(f"SELECT COUNT(*) as c FROM nessie.silver.{table}").collect()
            print(f"  ✅ Snapshots: {snapshots[0]['c']}")
            print(f"  ✅ load_date: {'SÍ' if any('load_date' in str(r) for r in schema) else 'NO'}")
            print(f"  ✅ Registros: {count[0]['c']}")
        except Exception as e:
            print(f"  ❌ Error: {e}")


def test_merge_functionality():
    """Verifica que el MERGE funciona"""
    print("\n" + "="*60)
    print("PRUEBA DE MERGE")
    print("="*60)
    for table in ["comments", "badges", "users", "posts"]:
        print(f"\n📋 {table}:")
        try:
            snapshots = spark.sql(f"""
                SELECT snapshot_id, committed_at, operation,
                       summary['added-records'] as added,
                       summary['total-records'] as total
                FROM nessie.silver.{table}.snapshots
                ORDER BY committed_at
            """)
            count = snapshots.count()
            print(f"  📸 Snapshots: {count}")
            if count > 1:
                print(f"  ✅ MERGE funcionando")
            else:
                print(f"  ⚠  Ejecuta nuevamente para probar MERGE")
            snapshots.show(truncate=False)
        except Exception as e:
            print(f"  ❌ Error: {e}")


def process_silver_layer_complete_fixed():
    """Proceso Silver completo"""
    print("=== PROCESO SILVER ===")
    setup_nessie_namespaces()
    
    print("\n--- COMMENTS ---")
    silver_comments = transform_multiple_years()
    merge_into_silver_table_compliant(silver_comments, "comments", ["comment_id"])
    
    print("\n--- OTROS DATASETS ---")
    transform_other_datasets()
    
    print("\n--- VERIFICACIÓN ---")
    verify_silver_compliance()
    
    print("\n=== COMPLETADO ===")
    return silver_comments


# Ejecutar
process_silver_layer_complete_fixed()
test_merge_functionality()

=== PROCESO SILVER ===
=== CONFIGURANDO NAMESPACES EN NESSIE ===
Namespaces existentes:
+---------+
|namespace|
+---------+
+---------+

Creando namespace 'silver'...
✅ Namespace 'silver' creado exitosamente
Namespaces después de la creación:
+---------+
|namespace|
+---------+
|   silver|
+---------+


--- COMMENTS ---
Procesando: manual - comments 2020...


25/10/15 21:17:14 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Leyendo de: s3a://lakehouse/bronze/Comments_2020.parquet
📑 Columnas normalizadas: ['id', 'post_id', 'score', 'text', 'creation_date', 'user_id', 'user_display_name']
✅ Leídos 1000 registros de s3a://lakehouse/bronze/Comments_2020.parquet


  ✅ manual 2020: 1000 registros
Procesando: dlt - comments 2021...
✅ Encontrado archivo DLT: 1760554762.3482103.95c5ec2953.parquet
Leyendo de: s3a://lakehouse/bronze/comments_2021/comments_2021/1760554762.3482103.95c5ec2953.parquet
📑 Columnas normalizadas: ['id', 'post_id', 'score', 'text', 'creation_date', 'user_id', 'user_display_name']
✅ Leídos 1000 registros de s3a://lakehouse/bronze/comments_2021/comments_2021/1760554762.3482103.95c5ec2953.parquet


  ✅ dlt 2021: 1000 registros
Realizando MERGE en: nessie.silver.comments
ℹ  Tabla nessie.silver.comments no existe, se creará
Creando nueva tabla Iceberg: nessie.silver.comments
